In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from extract_pvat_marker import *
import os
from extract_pvat_marker import periaortic_fat_marker, calculate_fat_volume_in_ml
from extract_pvat_marker import *

In [2]:
paths = Paths("./AC_CT", "./TEST_NIFTI", "./TEST_SEG", "./res.csv", "temp")

In [3]:
paths.create_missing_directories()

In [4]:
periaortic_fat_marker(paths)

Created missing_directories.....
Chris Rorden's dcm2niiX version v1.0.20211006  (JP2:OpenJPEG) GCC11.2.0 x86-64 (64-bit Linux)
Found 324 DICOM file(s)
Convert 324 DICOM as /home/chris/Desktop/NUKMED/TEST_NIFTI/temp/AC_CT_PET_CT_F18_FDG_venoes_20180618111929_2 (512x512x324x1)
Conversion required 0.788312 seconds (0.788011 for core code).
converreted dicom to nifti.....
Moved .....
Created isotropic....

If you use this tool please cite: https://pubs.rsna.org/doi/10.1148/ryai.230024

No GPU detected. Running on CPU. This can be very slow. The '--fast' or the `--roi_subset` option can help to reduce runtime.
Generating rough body segmentation...
Resampling...
  Resampled in 8.94s
Predicting...


/home/chris/anaconda3/envs/chris/lib/python3.12/site-packages/nnunetv2/utilities/plans_handling/plans_handler.py:37: UserWarning: Detected old nnU-Net plans format. Attempting to reconstruct network architecture parameters. If this fails, rerun nnUNetv2_plan_experiment for your dataset. If you use a custom architecture, please downgrade nnU-Net to the version you implemented this or update your implementation + plans.
  warnings.warn("Detected old nnU-Net plans format. Attempting to reconstruct network architecture "
100%|██████████| 45/45 [00:28<00:00,  1.56it/s]


  Predicted in 37.18s
Resampling...
  cropping from (512, 512, 710) to (213, 156, 270)
Resampling...
  Resampled in 2.24s
Predicting part 1 of 2 ...


/home/chris/anaconda3/envs/chris/lib/python3.12/site-packages/nnunetv2/utilities/plans_handling/plans_handler.py:37: UserWarning: Detected old nnU-Net plans format. Attempting to reconstruct network architecture parameters. If this fails, rerun nnUNetv2_plan_experiment for your dataset. If you use a custom architecture, please downgrade nnU-Net to the version you implemented this or update your implementation + plans.
  warnings.warn("Detected old nnU-Net plans format. Attempting to reconstruct network architecture "
100%|██████████| 12/12 [00:53<00:00,  4.48s/it]


Predicting part 2 of 2 ...


100%|██████████| 12/12 [00:53<00:00,  4.45s/it]


  Predicted in 121.59s
Resampling...
Saving segmentations...


100%|██████████| 4/4 [00:04<00:00,  1.08s/it]


  Saved in 9.42s


Start real code


TypeError: create_obj_organ_names() missing 1 required positional argument: 'organs_to_names'

In [4]:
organs_to_names = {"aorta":"aorta.nii.gz", 
                       "lung_ll":"lung_lower_lobe_left.nii.gz", 
                       "lung_lr":"lung_lower_lobe_right.nii.gz", 
                       "heart":"heart.nii.gz"}

In [5]:
organs:Dict[str, Organ] = create_obj_organ_names(paths, organs_to_names)
organs["isotropic"] = Organ("isotropic", paths.isotropic_nifti_path)

In [6]:

for organ in organs.keys():
    organs[organ].load_image()
    organs[organ].load_array()

organs["aorta"].load_nifti_options()
organs["isotropic"].load_nifti_options()

In [7]:
last_lung_z_index = get_lastZ_of_all([organs["lung_ll"].mask, organs["lung_lr"].mask])

In [10]:
def split_organ_z(z_value:int, organ_mask:np.ndarray):

    """ 
    Returns the part of the organ that's above a value on z-axis.
    """
    new = organ_mask.copy()
    new[:, :, z_value:] = 1
    new[:, :, :z_value] = 0
    return new * organ_mask

In [8]:
thoracic_aorta_mask = split_organ_z(last_lung_z_index, organs["aorta"].mask)
print("Start real code")

Start real code


In [9]:
dilation_length_mm=5
value_range=(-195, -45)
min_connected_voxels=1

In [13]:
save_nifti(thoracic_aorta_mask, organs, os.path.join(paths.segmentation_folder, f"thoracic_aorta.nii.gz"))

In [10]:
organs["thoracic_aorta"] = Organ("aorta", os.path.join(
                    paths.segmentation_folder, "thoracic_aorta.nii.gz"))

In [11]:
organs["thoracic_aorta"].load_array()
thoracic_aorta = organs["isotropic"].mask * organs["thoracic_aorta"].mask 


In [12]:
surrounding_mask = volume_around_mask(thoracic_aorta_mask, organs["isotropic"].nifti_options.pixdim, dilation_length_mm)
# save_nifti(surrounding_mask, organs, os.path.join(paths.segmentation_folder, f"thoracic_aorta_surrounding.nii.gz"))
thoracic_aorta_surrounding = organs["isotropic"].mask * surrounding_mask

In [13]:
heart = organs["isotropic"].mask * organs["heart"].mask
surrounding_mask = volume_around_mask(organs["heart"].mask, organs["isotropic"].nifti_options.pixdim, dilation_length_mm)
save_nifti(surrounding_mask, organs, os.path.join(paths.segmentation_folder, f"heart_surrounding.nii.gz"))

heart_surrounding = organs["isotropic"].mask * surrounding_mask

In [14]:
markers = pd.DataFrame()

target_volume_map = {
    "Periaortic": thoracic_aorta_surrounding,
    "Pericardial": heart_surrounding,
    "Aorta": thoracic_aorta,
    "Heart": heart,
}

for target_area in target_volume_map.keys():

    thres_vol_cm3, relative_vol = calculate_fat_volume_in_ml(
        target_volume_map[target_area], value_range,
        organs["isotropic"].nifti_options.isotropic_target_spacing, 
                                min_connected_voxels, 
                                paths, 
                                organs["aorta"].get_image()
                                )


    markers[f"{target_area} absolute volume (ml)"] = [thres_vol_cm3]
    markers[f"{target_area} relative volume"] = [relative_vol]

In [15]:
markers

,Periaortic absolute volume (ml),Periaortic relative volume,Pericardial absolute volume (ml),Pericardial relative volume,Aorta absolute volume (ml),Aorta relative volume,Heart absolute volume (ml),Heart relative volume
0,22.396833,0.127458,28.049707,0.114545,0.158444,0.000879,0.25811,0.000626
